INSTALLING GRADIO

In [1]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.6 MB/s et

IMPORTING LIBRARIES

In [4]:
import json
from difflib import get_close_matches
import gradio as gr

HELPER FUNCTIONS FOR MANIPULATING KNOWLEDGE BASE

In [5]:
def load_kb(filepath: str) -> dict:
    with open(filepath, 'r') as fp:
        data: dict = json.load(fp)
    return data

def save_kb(filepath: str, data: dict):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=2)

def update_kb(user_input: str, new_answer: str, kb: dict, kb_path: str):
    if new_answer.lower() != "skip":
        kb["questions"].append({"question": user_input, "answer": new_answer})
        save_kb(kb_path, kb)
        return "Thank you! I learned a new response!"
    else:
        return "No new response added."

INITIALIZING PATH FOR KNOWLEDGE BASE

In [6]:
# Path to the knowledge base
PATH = 'kb.json'
kb = load_kb(PATH)

FUNCTIONS FOR SEARCHING IN KB

In [7]:
def find_best_match(userqn: str, questions: list[str]):
    matches: list = get_close_matches(userqn, questions, n=1, cutoff=0.6)
    return matches[0] if matches else None

def get_answer(question: str, knowledge_base: dict):
    for q in knowledge_base["questions"]:
        if q["question"] == question:
            return q["answer"]
    return None

MAIN CHATBOT FUNCTIONS

In [8]:
def chatbot(user_input: str, kb: dict):
    bestmatch = find_best_match(user_input, [q["question"] for q in kb["questions"]])
    if bestmatch:
        answer = get_answer(bestmatch, kb)
        return answer
    else:
        return "I don't know the answer. Can you teach me?"

# Function to handle chat interaction
def chat_interaction(user_input, chat_history):
    bot_response = chatbot(user_input, kb)
    chat_history.append((user_input, bot_response))
    if bot_response == "I don't know the answer. Can you teach me?":
        return chat_history, gr.update(visible=True), gr.update(interactive=True)
    else:
        return chat_history, gr.update(visible=False), gr.update(interactive=True)

# Function to handle teaching the bot
def teach_bot(user_input, new_answer, chat_history):
    teach_response = update_kb(user_input, new_answer, kb, PATH)
    chat_history[-1] = (chat_history[-1][0], teach_response)  # Update last bot response
    return chat_history, gr.update(visible=False), gr.update(interactive=True)

MAIN GUI LOOP

In [10]:
# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# My Chatbot")

    initial_greeting = "Hello! How can I help you today?"
    chat_history = gr.State([(None, initial_greeting)])  # Initialize with a greeting message
    chat_log = gr.Chatbot(value=[(None, initial_greeting)])  # Set the initial value for the chat log
    user_input = gr.Textbox(show_label=False, placeholder="Type your question here...")
    new_answer = gr.Textbox(show_label=False, placeholder='Type the answer or "skip" to skip...', visible=False)

    def submit_interaction(user_input, chat_history):
        chat_history, new_answer_visible, user_input_interactive = chat_interaction(user_input, chat_history)
        return chat_history, new_answer_visible, user_input_interactive, ""

    def submit_teach(user_input, new_answer, chat_history):
        chat_history, new_answer_visible, user_input_interactive = teach_bot(user_input, new_answer, chat_history)
        return chat_history, new_answer_visible, user_input_interactive, ""

    user_input.submit(submit_interaction, inputs=[user_input, chat_history], outputs=[chat_log, new_answer, user_input])
    new_answer.submit(submit_teach, inputs=[user_input, new_answer, chat_history], outputs=[chat_log, new_answer, user_input])

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b7ff343cc0210d4e37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


SINCE THE LIVE UI DEMO IS NOT VISIBLE IN THE NOTEBOOK FILE, THE OUTPUT SCREENSHOTS ARE UPLOADED IN THE REPO